# Preguntas

In [ ]:
Responda a cada de las siguientes preguntas de forma clara y lo más completamente posible. 
1. Considera un modelo de regresión lineal con dos características, X₁ y X₂, y sus pesos correspondientes w₁ y w₂. Si el modelo predice una salida y mediante la ecuación y = 2w₁X₁ + 3w₂X₂ + 1, ¿cuál es la interpretación del coeficiente 3w₂ en el contexto del modelo? 
Teniendo en consideración que (w1,w2) son los respectivos pesos para (X1, X2) entonces tenemos que el coeficiente 3w2 es el aporte que tiene X2 para la salida Y. Es decir, qué tanta contribución tiene el peso w2 de la característica X2 en el resultado Y. 

2. Explica el concepto de multicolinealidad en el contexto de la regresión lineal. ¿Cómo afecta la multicolinealidad a la interpretación de los coeficientes de regresión individuales? 

La multicolinealidad es cuando en un modelo de regresión lineal dos o más variables independientes tienen correlación. Esto resulta en dificultad en la interpretación de coeficientes debido a que aumenta el error estándar , lo cual hace lo hace tener más incertidumbre en los datos. 


In [108]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np



In [109]:
df = pd.read_csv("dataset_phishing.csv", sep = ",")


## Encoding para la variable Status

In [110]:
# Crear columnas separadas para "legitimate" y "phishing" con valores 0 o 1
df['legitimate'] = (df['status'] == 'legitimate').astype(int)
df['phishing'] = (df['status'] == 'phishing').astype(int)

## Regresión logistica

### 1.) Procesar los datos

In [111]:
X = df[['length_url', 'nb_dots']].values  # Características
y = df['phishing'].values  # Variable objetivo

In [112]:
from sklearn.model_selection import train_test_split

# X = variable independiente
# Y = indica si la instancia es pishing (1) o (0)
# train_test_split: divide los datos en entrenamiento y prueba
# X_train, Y_train: despues de dividir los datos contiene el conjunto de entrenamiento de x & y
# X_test, Y_test: despues de dividir los datos contiene el conjunto de prueba de x & y 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


##### Colocamos la formula sigmoide de regresión logistica en una función para pasar los parametros

In [113]:
def sigmoide(z):
    return 1 / (1 + np.exp(-z))

##### Inicializar parametros w y b

In [114]:
def inicializarParams(dim):
    w = np.zeros((dim, 1)) #matriz de ceros
    b = 0 #inicializar el sesgo en 0 
    return w, b


In [115]:
def propagate(w, b, X, y): #w pesos, b sesgo, x matriz de fila representada A, y matriz de fila representada B 
    m = X.shape[0]
    
    # Forward: Se crea el sigmoide, calculando la combinación lineal
    A = sigmoide(np.dot(X, w) + b) 
    # Etropia cruzada: Mide el rendimiento del problema 
    #Al utilizar esta formula obtenemos el coste promedio por todas las instancias
    cost = (-1 / m) * np.sum(y * np.log(A + 1e-9) + (1 - y) * np.log(1 - A + 1e-9))
    
    # Backward:  utilizamos los parametros inicializados anteriromente (w y b) calculando los gradientes
    dw = (1 / m) * np.dot(X.T, (A - y)) #actualiza  los pesos en la dirección que reduce el coste
    db = (1 / m) * np.sum(A - y) #gradiante del coste con respecto al sesgo
    
    gradiantes = {"dw": dw, "db": db} #Diccionario con los gradiantes de los parametros utilizados (w y b)
    
    return gradiantes, cost


##### Encontrar los valores optimos para w y b

In [116]:
# iterations_optimize: Optimizar las iteraciones del conjunto de datos
# learning_rate: Curva de aprendizaje (tamaño de gradiante descendente)
# print_cost: si es true imprime el coste de cada 100 iteraciones
def optimize(w, b, X, y, iterations_optimize, learning_rate, print_cost=False):
    costs = []  # Lista para almacenar el valor del coste en cada iteración
    
    for i in range(iterations_optimize):
        gradients, cost = propagate(w, b, X, y)  # Obtiene los gradientes y el coste actual
        
        dw = gradients["dw"]
        db = gradients["db"]
        
        w = w - learning_rate * dw  # Actualiza los pesos
        b = b - learning_rate * db  # Actualiza el sesgo
        
        # Guarda el coste cada 100 iteraciones
        if i % 100 == 0:
            costs.append(cost)  # Usa 'costs.append' en lugar de 'cost.append'
            if print_cost:
                print(f"Costo despues de la iteración {i}: {cost}")
    
    params = {"w": w, "b": b}
    gradients = {"dw": dw, "db": db}
    
    return params, gradients, costs

    
    #Retorna los parametros, cost y gradiantes
    return params, gradiantes, cost


##### Por ultimo se predicen los nuevos datos X

In [117]:
def predict(w, b, X): # x es donde iran los nuevos datos
    m = X.shape[0] #num de ejemplos
    y_prediccion = np.zeros((m, 1)) #Array de ceros donde se almacenaran las predicciones
    w = w.reshape(X.shape[1], 1)
    # calcula la probabilidad de que cada X pertenezca a la clase 1 utilizando sigmoide
    A = sigmoide(np.dot(X, w) + b)
    #asigna una predicción de 1 si la probabilidad es mayor que 0.5, y 0 si no es asi
    for i in range(A.shape[0]):
        y_prediccion[i, 0] = 1 if A[i, 0] > 0.5 else 0
        
    return y_prediccion


## Entrenamiento del modelo

In [ ]:
# Inicializar parámetros
w, b = inicializarParams(X_train.shape[1])

# Gradiente descendente
iterations_optimize = 2000  # Número de iteraciones
iterations_optimize = 2500  # Número de iteraciones intento 2

learning_rate = 0.01
params, gradiantes, cost  = optimize(w, b, X_train, y_train, iterations_optimize, learning_rate, print_cost=True)

# Predicciones
w = params["w"]
b = params["b"]
y_pred_train = predict(w, b, X_train)
y_pred_test = predict(w, b, X_test)


Costo despues de la iteración 0: 6338.137800753336
Costo despues de la iteración 100: 1.747763484617389
Costo despues de la iteración 200: 1.6214618643575636
Costo despues de la iteración 300: 1.513158300248424
Costo despues de la iteración 400: 1.4189940869484645


##### Graficar el costo

In [ ]:
import matplotlib.pyplot as plt

plt.plot(costs)
plt.ylabel('Costo')
plt.xlabel('Iteraciones (por cientos)')
plt.title('Costo vs. Número de Iteraciones')
plt.show()


In [ ]:
# Boxplot
plt.figure(figsize=(8, 5))
sns.boxplot(x='status', y='length_url', data=df, palette='pastel')
plt.title('Boxplot de Longitud de la URL por Clase')
plt.xlabel('Clase')
plt.ylabel('Longitud de la URL')
plt.show()


In [ ]:
# Histograma para Legitimate
plt.figure(figsize=(12, 5))
sns.histplot(legitimate_data['nb_dots'], bins=15, kde=True, color='lightcoral')
plt.title('Histograma de Cantidad de Puntos en la URL para Legitimate')
plt.xlabel('Cantidad de Puntos')
plt.ylabel('Frecuencia')
plt.show()


In [ ]:

# Histograma para Phishing
plt.figure(figsize=(12, 5))
sns.histplot(phishing_data['nb_dots'], bins=15, kde=True, color='skyblue')
plt.title('Histograma de Cantidad de Puntos en la URL para Phishing')
plt.xlabel('Cantidad de Puntos')
plt.ylabel('Frecuencia')
plt.show()
